In [35]:
import pandas_datareader.data as web
import pandas as pd
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import importlib
import plotly.express as px

#Change me!
start = datetime.datetime.today() + relativedelta.relativedelta(years=-10)
#Mで入力する
split_unit = "12M"
modulename = "BacktestingStrategies.Strategy_RsiOscillator"
classname = "btst.RsiOscillator"
end = datetime.date.today()

data = web.DataReader('2558.T', 'yahoo', start, end)
data = data.astype("double")

initial = 1000_000
fname = f"html/plot-"

In [36]:
#日付データを検証期間単位ごとにSplit
#https://kakakakakku.hatenablog.com/entry/2021/05/24/002542
grouped_data = data.groupby(pd.Grouper(freq = split_unit))
dfs = [group for _, group in grouped_data]

In [37]:
#検証期間単位ごとにバックテスト＆結果を蓄積
#親ディレクトリ経由で呼び出し   # https://qiita.com/yokohama4580/items/466a483ae022d264c8ee
import os
import sys
import re
sys.path.append(os.pardir)

btst = importlib.import_module(modulename)
# from BacktestingStrategies import Strategy_RsiOscillator as btst
importlib.reload(btst)
import warnings
warnings.simplefilter('ignore')

warmup = int(re.sub(r"\D", "", split_unit)) / 2 * 20

returns = {}
bts = {}
i = 0

for monthly_data in dfs:
    if i != 0:
        start = dfs[i - 1].tail(int(warmup))
        end = data[data.index == monthly_data.tail(1).index.tolist()[0]]
        monthly_data = pd.concat([start, end])

    
    # バックテストを設定
    bt = Backtest(
        monthly_data,
        eval(classname), # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    returns[bt] = output
    # bts[str(i)] = bt

    i = i + 1

In [38]:
#ベスト１、ワースト１、ワースト３を抽出
returns_without0trades = {k: v for k, v in returns.items() if v["# Trades"] != 0}
returns_sorted = sorted(returns_without0trades.items(), key=lambda x: x[1]["Return [%]"])

min_return_3 = returns_sorted[:3]
top1_return = returns_sorted[-1:]

max_return = str((top1_return[0][1])["Return [%]"])
max_period_start = str((top1_return[0][1])["Start"])
max_period_end = str((top1_return[0][1])["End"])

min_return = str((min_return_3[0][1])["Return [%]"])
min_period_start = str((min_return_3[0][1])["Start"])
min_period_end = str((min_return_3[0][1])["End"])

In [39]:
#まとめを表示
import statistics
df = pd.DataFrame(returns.values())

lines = "１期間単位: " + split_unit + "\n"
lines += "テスト期間: " + str(len(returns)) + "\n"
res = round(statistics.mean(df["Return [%]"]), 4)

lines += "平均Return [%]: " + str(res) + "\n"
lines += "最大Return [%]: " + str(max_return) + "\n"
lines += "最大Return期間: " + str(max_period_start) + " - " + str(max_period_end) + "\n"
lines += "最低Return [%]: " + str(min_return) + "\n"
lines += "最低Return期間: " + str(min_period_start) + " - " + str(min_period_end) 
print(lines)

１期間単位: 12M
テスト期間: 4
平均Return [%]: 6.1311
最大Return [%]: 12.676950000000001
最大Return期間: 2020-08-04 00:00:00 - 2022-01-31 00:00:00
最低Return [%]: 11.84742649999999
最低Return期間: 2021-08-05 00:00:00 - 2022-10-18 00:00:00


In [40]:
#ワースト３を表示
i = 1
for min_data in min_return_3:
    print("----- WORST " + str(i) + "-----")
    display(min_data[1])
    display(min_data[1]._trades)
    min_data[0].plot(filename = fname + str(min_data[1]._strategy) + "_min" + str(i) + ".html")

    i = i + 1

----- WORST 1-----


Start                     2021-08-05 00:00:00
End                       2022-10-18 00:00:00
Duration                    439 days 00:00:00
Exposure Time [%]                   15.702479
Equity Final [$]                  1118474.265
Equity Peak [$]                   1118474.265
Return [%]                          11.847426
Buy & Hold Return [%]                14.61039
Return (Ann.) [%]                    26.26142
Volatility (Ann.) [%]               16.993041
Sharpe Ratio                         1.545422
Sortino Ratio                        5.531121
Calmar Ratio                          6.81878
Max. Drawdown [%]                   -3.851337
Avg. Drawdown [%]                   -3.851337
Max. Drawdown Duration       20 days 00:00:00
Avg. Drawdown Duration       20 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                        8.32757
Worst Trade [%]                      3.296861
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,70,34,50,14250.1910,14720.0,32886.630,0.032969,2021-09-27,2021-10-19,22 days
1,70,119,120,14682.3195,15905.0,85587.635,0.083276,2022-01-31,2022-10-18,260 days


----- WORST 2-----


Start                     2020-08-04 00:00:00
End                       2022-01-31 00:00:00
Duration                    545 days 00:00:00
Exposure Time [%]                   58.677686
Equity Final [$]                    1126769.5
Equity Peak [$]                     1128749.5
Return [%]                           12.67695
Buy & Hold Return [%]               47.347347
Return (Ann.) [%]                   28.219493
Volatility (Ann.) [%]               17.939474
Sharpe Ratio                         1.573039
Sortino Ratio                        3.536201
Calmar Ratio                         3.090181
Max. Drawdown [%]                   -9.131986
Avg. Drawdown [%]                   -1.915424
Max. Drawdown Duration      388 days 00:00:00
Avg. Drawdown Duration       44 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      12.741927
Worst Trade [%]                     12.741927
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,99,36,106,10049.5,11330.0,126769.5,0.127419,2020-09-28,2021-01-12,106 days


In [41]:
#ベストを表示
print("----- BEST -----")
display(top1_return[0][1])
display(top1_return[0][1]._trades)
top1_return[0][0].plot(filename = fname + str(top1_return[0][1]._strategy) + "_best.html")

----- BEST -----


Start                     2020-08-04 00:00:00
End                       2022-01-31 00:00:00
Duration                    545 days 00:00:00
Exposure Time [%]                   58.677686
Equity Final [$]                    1126769.5
Equity Peak [$]                     1128749.5
Return [%]                           12.67695
Buy & Hold Return [%]               47.347347
Return (Ann.) [%]                   28.219493
Volatility (Ann.) [%]               17.939474
Sharpe Ratio                         1.573039
Sortino Ratio                        3.536201
Calmar Ratio                         3.090181
Max. Drawdown [%]                   -9.131986
Avg. Drawdown [%]                   -1.915424
Max. Drawdown Duration      388 days 00:00:00
Avg. Drawdown Duration       44 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      12.741927
Worst Trade [%]                     12.741927
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,99,36,106,10049.5,11330.0,126769.5,0.127419,2020-09-28,2021-01-12,106 days


Row(id='21118', ...)

In [42]:
#散布図を表示
fig = px.box(df["Return [%]"], points = "all")
fig.update_layout(
    xaxis_title = "Strategy",
    yaxis_title = "Return [%]"
)
fig.show()